In [1]:
import pandas as pd
import numpy as np
import jieba

In [2]:
from cleanse import separateUserContent,separateAllContent
from random import shuffle, randint
from gensim.models import word2vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense ,SimpleRNN, Embedding, Activation, LSTM, GRU,Bidirectional,GlobalMaxPooling1D,Dot,Lambda,Add,Conv1D
from keras.models import Model,  Sequential, load_model,model_from_json
from keras import backend as K
from keras.callbacks import ModelCheckpoint

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
tmp_tup_df =pd.read_pickle("validation_tup_list.pkl")#固定TEST ANS SET讓測試資料都一樣
validation_tup_list =tmp_tup_df.values.tolist()
print(validation_tup_list[0])

['我的T102H一直重開機不能用 平板電腦説找筆電處理', '電源有重新插拔過嗎 有等他跑完過嗎']


In [4]:
tmp_tup_df =pd.read_pickle("training_tup_list.pkl")#固定TEST ANS SET讓測試資料都一樣
training_tup_list =tmp_tup_df.values.tolist()
print(training_tup_list[0])

['鬧鐘無法關閉 智能管家無封鎖', '您說無法關閉是  請問有畫面嗎']


In [5]:
print(len(training_tup_list))

49916


In [6]:
vec_model = word2vec.Word2Vec.load("word2vec.model")

In [7]:
import pickle
f = open("word_index_dic.pkl","rb")
word_index = pickle.load(f)
f.close()

In [8]:
word_index[" "]

2820

In [9]:
def makeQuesFeatuers(vec_model,word_index):
    docs = []

   
            
    for i in range(len(training_tup_list)):
        list1 = jieba.cut(training_tup_list[i][0], cut_all = False)
    
        docs.append(list(list1))

   
   

    encoded_docs = []
    for list10 in docs:
        list_temp = []
        for word in list10:
            if word in vec_model.wv.vocab:
                list_temp.append(word_index[word])
        encoded_docs.append(list_temp)

    #print(encoded_docs[0])
    max_length = 40
    padded_docs_list = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

    padded_docs = np.array(padded_docs_list)

    return padded_docs

In [10]:
train_q_fea = makeQuesFeatuers(vec_model, word_index)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.691 seconds.
Prefix dict has been built succesfully.


In [11]:
max_len = 40 #model的部份主要按照ANSWER SELECTION GITHUB
input_q = Input((max_len,))


In [12]:
embedding_matrix = np.zeros((len(vec_model.wv.vocab)+1, 250))
for word, i in word_index.items():
        embedding_vector = vec_model.wv[word]
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(vec_model.wv.vocab)+1, 250, weights = [embedding_matrix], input_length = 40)

In [13]:
emb_q = embedding_layer(input_q)


In [14]:
shared_lstm = Bidirectional(
    LSTM(300, return_sequences=True)
)
encode_q = shared_lstm(emb_q)

In [15]:
vec_q = GlobalMaxPooling1D()(encode_q)

In [16]:
result = Dense(49916)(vec_q)

In [17]:
model = Model(inputs = input_q, outputs = result)

In [18]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 40, 250)           1716250   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40, 600)           1322400   
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 600)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 49916)             29999516  
Total params: 33,038,166
Trainable params: 33,038,166
Non-trainable params: 0
_________________________________________________________________


In [20]:
label = np.zeros((49916,49916))
for i in range(0, 49916):
    label[i][i] = 1


In [21]:
filepath="weights_best_49916.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1, save_best_only=True,save_weights_only=False, mode='min')
callbacks_list = [checkpoint]

In [22]:
model.fit(train_q_fea, label , callbacks=callbacks_list,epochs= 3, batch_size = 256, verbose = 1)

Epoch 1/3
49916/49916 [==============================] - 43s 853us/step - loss: 2.6224e-05

Epoch 00001: loss improved from inf to 0.00003, saving model to weights_best_49916.hdf5
Epoch 2/3
49916/49916 [==============================] - 37s 742us/step - loss: 2.0749e-05

Epoch 00002: loss improved from 0.00003 to 0.00002, saving model to weights_best_49916.hdf5
Epoch 3/3
49916/49916 [==============================] - 37s 739us/step - loss: 2.0518e-05

Epoch 00003: loss improved from 0.00002 to 0.00002, saving model to weights_best_49916.hdf5


In [23]:
model.load_weights(filepath ="weights_best_49916.hdf5" )

In [24]:
model_json = model.to_json()
with open("model_49916_01.json", "w") as json_file:
    json_file.write(model_json)

In [25]:
def makeTestQuesFeatuers(vec_model,word_index):
    docs = []

   
            
    for i in range(len(validation_tup_list)):
        list1 = jieba.cut(validation_tup_list[i][0], cut_all = False)
    
        docs.append(list(list1))

   
   

    encoded_docs = []
    for list10 in docs:
        list_temp = []
        for word in list10:
            if word in vec_model.wv.vocab:
                list_temp.append(word_index[word])
        encoded_docs.append(list_temp)

    #print(encoded_docs[0])
    max_length = 40
    padded_docs_list = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

    padded_docs = np.array(padded_docs_list)

    return padded_docs

In [26]:
test_q_fea = makeTestQuesFeatuers(vec_model, word_index)

In [71]:
ans = model.predict(test_q_fea)

In [72]:
ans_max =[]
ans_num = []

for i in range(0,5):
    temp_ind = np.argmax(ans, axis = 1)
    temp_num = np.max(ans,axis = 1)
    ans_num.append(temp_num)
    ans_max.append(temp_ind)
    for j in range(0,len(ans)):
        ans[j][temp_ind[j]] = -100
    
        
        
        

In [94]:
ans_max[5][0]

IndexError: list index out of range

In [88]:
ans_num[2][10]

0.0006934507

In [80]:
with open("Output_49916.txt", "w") as text_file:
    for i in range(len(validation_tup_list)):
        text_file.write("Question %d : %s\n" % (i+1, validation_tup_list[i][0]))
        text_file.write("Ans: 0. %s %f\n" % (training_tup_list[ans_max[0][i]][1], ans_num[0][i]))
        text_file.write("     1. %s %f\n" % (training_tup_list[ans_max[1][i]][1], ans_num[1][i]))
        text_file.write("     2. %s %f\n" % (training_tup_list[ans_max[2][i]][1], ans_num[2][i]))
        text_file.write("     3. %s %f\n" % (training_tup_list[ans_max[3][i]][1], ans_num[3][i]))
        text_file.write("     4. %s %f\n" % (training_tup_list[ans_max[4][i]][1], ans_num[4][i]))
        text_file.write("True answer is \n")
        text_file.write("Ans: %s\n" % validation_tup_list[i][1])
        text_file.write("===========================\n")